In [1]:
import requests as req
from bs4 import BeautifulSoup
import bs4
import getpass
import os

os.environ["MISTRAL_API_KEY"] = getpass.getpass()
os.environ['HF_TOKEN'] = getpass.getpass()

# get all links

In [4]:
url = 'https://www.gamespot.com/games/reviews/?review_filter_type%5Bplatform%5D=&review_filter_type%5Bgenre%5D=&review_filter_type%5BtimeFrame%5D=P12M&review_filter_type%5BstartDate%5D=&review_filter_type%5BendDate%5D=&review_filter_type%5BminRating%5D=6&review_filter_type%5Btheme%5D=&review_filter_type%5Bregion%5D=&___review_filter_type%5Bpublishers%5D=&___review_filter_type%5Bdevelopers%5D=&review_filter_type%5Bletter%5D=&sort=date&page='
links = []

In [5]:
def parse_link(url):
    links = []
    response = req.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    parse_only=bs4.SoupStrainer(
        class_=('card-item__link text-decoration--none')
    )
    a = soup.find_all(parse_only)
    for link in a:
        links.append('https://www.gamespot.com'+link.get('href'))
    return links

In [6]:
for i in range(1,7):
    links+=parse_link(url+str(i))

In [8]:
def create_name(url):
    return 'The following piece is about the game: '+' '.join(url[url.find('com/reviews')+len('com/reviews/'):url.find('-review')].split('-')).capitalize()+ '. '

In [9]:
links.pop(-6)

'https://www.gamespot.com/reviews/another-code-recollection-full-of-mysteries/1900-6418168/'

In [11]:
len(links)

112

# parse split embed

In [13]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_mistralai import MistralAIEmbeddings



def embed(url):

    name = create_name(url)
    loader = WebBaseLoader(web_paths=[url],  bs_kwargs={'parse_only': bs4.SoupStrainer('p')})
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300, add_start_index=True)
    
    all_splits = text_splitter.split_documents(loader.lazy_load())
    
    all_docs = [name + i.page_content for i in all_splits]

    embeddings = MistralAIEmbeddings(model="mistral-embed")
    text_embeddings = embeddings.embed_documents(all_docs)
    text_embedding_pairs = zip(all_docs, text_embeddings)
    
    return text_embedding_pairs

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [15]:
embedding_model = MistralAIEmbeddings(model="mistral-embed")

C:\Users\matve\anaconda3\Lib\site-packages\langchain_mistralai\embeddings.py:175: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [16]:
db = FAISS.from_embeddings(embed(links[0]),embedding=embedding_model)

In [17]:
i = 1
while i<len(links):
    try:
        db.add_embeddings(embed(links[i]))
        i+=1
    except:
        print(i)
        continue

An error occurred with MistralAI: 'data'


1


An error occurred with MistralAI: 'data'


1


An error occurred with MistralAI: 'data'


...

97


An error occurred with MistralAI: 'data'


97


An error occurred with MistralAI: 'data'


105


In [18]:
db.index.ntotal

1220

In [19]:
db.save_local('faiss_storage_1500_300')

# test

In [22]:
db.similarity_search('call of duty')

[Document(metadata={}, page_content="The following piece is about the game: Call of duty black ops 6 campaign. where you just march forward and shoot guys in an approach that harkens all the way back to the beaches of Normandy and the first Call of Duty, there are also plenty of other moments where you need to be sneaky and careful. Your mission to bring aboard master of disguise Sev is a low-profile trip through mob enforcers, in which you listen in on a phone call and then assassinate an underling. You're more or less free to figure out your way into a well-guarded church belltower to set up for the shot. You can blast your way in or stealth your way through, and while the level isn't huge or expansive in an immersive-sim way, there are enough options that your choice of how to approach the problem feels meaningful.There are a bunch of opportunities for stealth like this. Black Ops 6 isn't great as a stealth game--enemies are dumb as hell, their peripheral vision is basically nonexis